In [4]:
import pandas as pd
import numpy as np
import sys, json, os, io
from tqdm import tqdm
import distutils.dir_util
from collections import Counter

In [18]:
class naiveknn:
    
    def __init__(self, play_list=None, val_list=None, song_meta=None, debug=False):
        train = pd.read_json(play_list, encoding = 'utf-8')
        val = pd.read_json(val_list, encoding = 'utf-8')
        self.train_songs = train.songs
        self.train_tags = train.tags
        self.val_tags = val.tags
        self.val_songs = val.songs
        self.ids = val.id
        

        self.debug = debug
        if debug: pass
        else: self.predict()
        
    def write_json(self, data, fname):
        def _conv(o):
            if isinstance(o, (np.int64, np.int32)):
                return int(o)
            raise TypeError
        
        parent = os.path.dirname(fname)
        distutils.dir_util.mkpath("./" + parent)
        
        with io.open("./" + fname, "w", encoding="utf-8") as f:
            json_str = json.dumps(data, ensure_ascii=False, default=_conv)
            f.write(json_str)

    def calculate(self, i, k=100, top=100):
        
        V = [set(lst) for lst in self.train_songs]
        u = set(self.val_songs[i])
        z = set(self.val_tags[i])
        song_set = set()
        tag_set = set()
        while (len(song_set) < 100):
            S = np.array([len(u & v) for v in V])
            top_k = S.argsort()[-k:]
            
            song_list = []
            tag_list = []
            for v in top_k:
                song_list += self.train_songs[v]
                tag_list += self.train_tags[v]
                song_set = set(song_list) - u
            
            c=Counter(tag_list).most_common(30)
            tag_set = [a for a,b in c if a not in z]
            if len(song_set) <100:
                k = k + 100
        
        norm = S[top_k].sum()
        if norm == 0:
            norm = 1
        
        r_u = [(i, np.array([S[v] if i in V[v] else 0 for v in top_k]).sum() / norm) for i in song_set]
        result = sorted(r_u, key=(lambda x: x[1]))
        similar_songs = [x[0] for x in result[-top:]][::-1]
        
        similar_tags = tag_set[:10]

        return similar_songs, similar_tags
    
    
    def predict(self):
        result = []
        for i, v in tqdm(enumerate(self.val_songs)):
            a, b = self.calculate(i)
            result.append({
                "id" : int(self.ids[i]),
                "songs" : a,
                "tags" : b,
                
            })
        
        self.write_json(result, "results.json")

In [19]:
knn = naiveknn('train.json', 'val.json', 'song_meta.json')

23015it [11:06:48,  1.74s/it] 
